In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torch.nn.init as weight_init

In [2]:
class Encoder(nn.Module):
    """
    Class for Symmetrical AutoEncoder Network
    :param L: List of int, contains sizes of encoding layers and starts with feature size
    For example: [500, 20, 10] will result in:
      - encoder 2 layers: 500x20 and 20x10. Representation layer (z) will be 10
      - decoder 2 layers: 10x20 and 20x500, output size is 500, reconstructed.
    :param activation_fn: (default 'sigmoid') Type of activation function
    :param drop_prob: (default: 0.0) Dropout probability
    """
    def __init__(self, L, activation_fn='sigmoid', drop_prob=0.0):
        super(Encoder, self).__init__()
        layers = self.create_nn_structure(L)
        self.num_layers = len(L)
        # create controller for activation function
        self.activation_fn_nm = activation_fn
        # create dropout module
        self._drop_prob = drop_prob
        if drop_prob > 0.0:
            self.dropout = nn.Dropout(drop_prob)
        #initialize with empty list to store layers
        self.linears = nn.ModuleList([])
        self.linears.extend([nn.Linear(i[0], i[1]) for i in layers])
        
    def get_activation_fn(self):
        # user selected activation function at layers except for last layer
        if self.activation_fn_nm == 'relu':
            return nn.ReLU()
        elif self.activation_fn_nm == 'lrelu':
            return nn.LeakyReLU()
        elif self.activation_fn_nm == 'sigmoid':
            return nn.Sigmoid()
        else:
            raise ValueError('Activation function type not defined')
    
    def forward(self, x):
        for i,layer in enumerate(self.linears):
            if i <= self.num_layers-1:
                # create instance of activation function
                act_fn = self.get_activation_fn()
                # pass in the input
                x = act_fn(self.linears[i](x))
                if self._drop_prob > 0.0 and i <= int(self.num_layers/2): 
                    # apply dropout only on encode layer by control of i
                    x = self.dropout(x)
        # No activation on the last decoding layer
        x = self.linears[-1](x)
        return x

    def create_nn_structure(self, L):
        max_ind = len(L)-1
        layers = []
        for i,v in enumerate(L):
            if i < max_ind:
                #still have i+1 available, create layer tuple
                layer = [v,L[i+1]]
                layers.append(layer)
        #then inverse the layers for decoder size
        encoder_layers = layers[:]
        for l in encoder_layers[::-1]:
            decoder_layer = l[::-1]
            layers.append(decoder_layer)
        return layers

In [3]:
def create_index_mapping(L):
    '''
    return reindexed dict on user and items
    encoded indices starts from 1
    input: 
    * L: list of str
    outputs:
    * ind_2_item,item_2_ind: tuple of dictionary
    '''
    L = set(L)
    ind_2_item = {}
    
    for i,v in enumerate(L):
        #index start from 1
        ind_2_item[i+1] = v
    #invert the map
    item_2_ind = {v: k for k, v in ind_2_item.items()}
    return ind_2_item,item_2_ind
    
def reindexer(ratings_df,user_col,item_col,rating_col):
    '''
    inputs:
    * ratings_df: pandas df containing ratings/affinity for user-item pairs
    * user_col: actual col name for users
    * item_col: actual col name for items
    * rating_col: actual col name for ratings
    output:
    * ratings_df: reindexed user and item column, pandas df
    '''
    users_list = ratings_df[user_col].tolist()
    item_list = ratings_df[item_col].tolist()
    
    ind_2_user,user_2_ind = create_index_mapping(users_list)
    ind_2_item,item_2_ind = create_index_mapping(item_list)
    
    #rename ratings df
    ratings_df = ratings_df.rename(columns={user_col:'user_col',
                                            item_col:'item_col',
                                            rating_col:'rating_col'})

    #encode df using the 2 mappings
    ratings_df['encoded_users'] = ratings_df['user_col'].apply(lambda x:user_2_ind[x])
    ratings_df['encoded_items'] = ratings_df['item_col'].apply(lambda x:item_2_ind[x])
    
    return ratings_df[['encoded_users','encoded_items','rating_col']]
  
ratings = pd.read_csv('MovieLens1M/ratings.dat', delimiter = '::',header=None, engine='python')
ratings.columns = ['userid','movieid','rating','timestamp']
ratings_reindex = reindexer(ratings,'userid','movieid','rating')


from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings_reindex,
                               stratify=ratings_reindex['encoded_users'],
                               test_size=0.1,
                               random_state=42)

training_set = np.array(train, dtype = 'int')
test_set = np.array(test, dtype = 'int')

nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

def convert(data):
    new_data = []
    for id_users in range(nb_users+1):
        # each user's watched movies
        # data[:,0], first column, all rows column users
        id_items = data[:,1][data[:,0] == id_users]
        # each user's rating for that item
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        # the positions of these items are filled with ratings, creating the matrix
        ratings[id_items-1] = id_ratings
        new_data.append(list(ratings))
    return new_data
  
training_set = convert(training_set)
test_set = convert(test_set)

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [4]:
autoencoder_network = Encoder([nb_movies,20,10],'sigmoid',0.1)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(autoencoder_network.parameters(), lr = 0.01, weight_decay = 0.5)

nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    # s is the number of users who rated at least 1 movies
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = autoencoder_network(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) #making this anyway not equal to 0, as this will be a denominator
            #mean_corrector is the avg of the error, only considering the movies having ratings (non-zero ratings) for computing mean of error
            loss.backward() # decide the direction the increment of weights
            #this call will just computing all the gradients required
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step() # decide the amount to update the weights
            
    print('epoch: '+str(epoch)+' loss: '+ str(train_loss/s))
    
    
test_loss = 0
s = 0.

res = []
targets = []

# averaged difference between real rating and predicted rating
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0) # should keep the training set
    target = Variable(test_set[id_user]).unsqueeze(0) # to predict the other movies user not seen yet
    
    if torch.sum(target.data > 0) > 0:
        # make predictions
        output = autoencoder_network(input)
        targets.append(target.detach().numpy())
        res.append(output.detach().numpy()) 
        target.require_grad = False
        output[target == 0] = 0 # dont want to measure the loss on the movies didnt get the actual rating from user 
        # force to 0 and difference / loss will be 0 for those entries
        loss = criterion(output, target)
        
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) 
        # only consider the movies that are rated in the test set, to be included in the loss
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

epoch: 1 loss: tensor(1.2801)
epoch: 2 loss: tensor(0.9990)
epoch: 3 loss: tensor(0.9853)
epoch: 4 loss: tensor(0.9805)
epoch: 5 loss: tensor(0.9783)
epoch: 6 loss: tensor(0.9769)
epoch: 7 loss: tensor(0.9765)
epoch: 8 loss: tensor(0.9757)
epoch: 9 loss: tensor(0.9756)
epoch: 10 loss: tensor(0.9749)
test loss: tensor(0.9524)


In [5]:
#making top k recommendation
def make_top_k_recommendations(encoder,evidence,k,filter_seen=True):
    '''
    :param encoder: autoencoder instance
    :param evidence: full set of seen ratings from all users
    :param k: top k items (by output score)
    :param filter_seen: (default True) filter controller to remove seen items from top k list
    '''     
    res = []
    nb_users = evidence.shape[0]
    # to find top scored items for each user
    for id_user in range(nb_users):
        encoder_input = Variable(evidence[id_user]).unsqueeze(0) # should keep the training set 
        encoder_output = encoder(encoder_input)
        
        target = Variable(evidence[id_user]).unsqueeze(0) # mask to find items not seen yet
        if filter_seen:
            encoder_output[target != 0] = 0 # force seen items scores to 0, will never get recommended
        res.append(encoder_output.detach().numpy())
        
    res = [a[0] for a in res]
    final_itemsets = []    
    for each in res:
        full_ratings_predicted = list(each)
        full_ratings_indexed = list(enumerate(full_ratings_predicted))
        final_itemsets.append(sorted(full_ratings_indexed,key=lambda x:x[1],reverse =True)[:k])
        
    return final_itemsets